# Lesson 5: Human in the Loop

In [2]:
from dotenv import load_dotenv

_ = load_dotenv()

In [32]:
from typing import TypedDict, Annotated, Optional, List
from uuid import uuid4

from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import StateGraph, END
from langgraph.checkpoint import BaseCheckpointSaver
from langgraph.pregel.types import StateSnapshot

from langchain_core.messages import (
    AnyMessage,
    SystemMessage,
    HumanMessage,
    ToolMessage,
    BaseMessage,
)
from langchain_core.language_models.chat_models import BaseChatModel
from langchain_core.tools import BaseTool
from langchain_anthropic import ChatAnthropic
from langchain_community.tools.tavily_search import TavilySearchResults

In [4]:
memory = SqliteSaver.from_conn_string(":memory:")
tool = TavilySearchResults(max_results=2)

## Manual Human Approval

In [5]:
"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""


def reduce_messages(
    left: list[AnyMessage], right: list[AnyMessage]
) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

In [6]:
#! Interrupt if other tools are called??

In [7]:
class Agent:
    def __init__(
        self,
        model: BaseChatModel,
        tools: List[BaseTool],
        checkpointer: Optional[BaseCheckpointSaver] = None,
        system: str = "",
    ) -> None:
        self.system = system

        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_llm)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(checkpointer=checkpointer, interrupt_before=["action"])

        self.tools = {tool.name: tool for tool in tools}
        self.model = model.bind_tools(tools)

    def exists_action(self, state: AgentState) -> bool:
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def call_llm(self, state: AgentState) -> dict[str, list[BaseMessage]]:
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def take_action(self, state: AgentState) -> dict[str, list[ToolMessage]]:
        tool_calls = state["messages"][-1].tool_calls
        results: list[ToolMessage] = []

        for t in tool_calls:
            print(f"Calling: {t}")
            if t["name"] not in self.tools:
                print("\n ....bad tool name....")
                result = "bad tool name, retry"
            else:
                result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")

        return {"messages": results}

In [8]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatAnthropic(model_name="claude-3-5-sonnet-20240620")
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

In [9]:
messages = [HumanMessage(content="Whats the weather in SF?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in San Francisco for you using the available search tool. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01QEhVrxFnemkZVoVZnjY5Ng', 'input': {'query': 'current weather in San Francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01UQgU8edpgLzufHeHYivcea', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 109}}, id='run-6ccbfbe7-e951-4aa5-a223-d85eaa1736e0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'toolu_01QEhVrxFnemkZVoVZnjY5Ng'}], usage_metadata={'input_tokens': 471, 'output_tokens': 109, 'total_tokens': 580})]}


In [10]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='2bc7ec98-8424-405c-b642-2563c44ae3c3'), AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in San Francisco for you using the available search tool. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01QEhVrxFnemkZVoVZnjY5Ng', 'input': {'query': 'current weather in San Francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01UQgU8edpgLzufHeHYivcea', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 109}}, id='run-6ccbfbe7-e951-4aa5-a223-d85eaa1736e0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'toolu_01QEhVrxFnemkZVoVZnjY5Ng'}], usage_metadata={'input_tokens': 471, 'output_tokens': 109, 

In [11]:
abot.graph.get_state(thread).next

('action',)

### Continue after Interrupt


In [12]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'toolu_01QEhVrxFnemkZVoVZnjY5Ng'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.wunderground.com/hourly/us/ca/san-francisco/94108/date/2024-07-10\', \'content\': \'Current Weather for Popular Cities . San Francisco, CA 58 ° F Cloudy; Manhattan, NY warning 80 ° F Fair; Schiller Park, IL (60176) warning 66 ° F Cloudy; Boston, MA warning 79 ° F Mostly ...\'}, {\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'San Francisco\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 37.78, \'lon\': -122.42, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1720620450, \'localtime\': \'2024-07-10 7:07\'}, \'current\': {\'last_updated_epoch\': 1720620000, \'last_updated\': \'2024-07-10 07:00\', \'temp_c\': 15.6, \'temp_f\': 60.1, \'is_day\': 1, \'condition\': {\'text\': \'Mist\', \'icon\':

In [13]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in SF?', id='2bc7ec98-8424-405c-b642-2563c44ae3c3'), AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in San Francisco for you using the available search tool. Let me do that for you now.", 'type': 'text'}, {'id': 'toolu_01QEhVrxFnemkZVoVZnjY5Ng', 'input': {'query': 'current weather in San Francisco'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01UQgU8edpgLzufHeHYivcea', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 109}}, id='run-6ccbfbe7-e951-4aa5-a223-d85eaa1736e0-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in San Francisco'}, 'id': 'toolu_01QEhVrxFnemkZVoVZnjY5Ng'}], usage_metadata={'input_tokens': 471, 'output_tokens': 109, 

In [14]:
abot.graph.get_state(thread).next

()

### All together

In [15]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)
while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread),"\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

{'messages': [AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in Los Angeles for you using the available search tool. Let me do that for you right away.", 'type': 'text'}, {'id': 'toolu_014MJQQWbWAuXhqzUZAKyswE', 'input': {'query': 'current weather in Los Angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_0164PLPdrqUQbzrwKcWZyqy7', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 110}}, id='run-28f4152c-386e-4d38-817f-edc7f8bf0b80-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'toolu_014MJQQWbWAuXhqzUZAKyswE'}], usage_metadata={'input_tokens': 471, 'output_tokens': 110, 'total_tokens': 581})]}

 StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id=

## Modify State

Run until the interrupt and then modify the state.

In [16]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

{'messages': [AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in Los Angeles for you using the available search tool. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb', 'input': {'query': 'current weather in Los Angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01KVfU8uRoFebxeWdcGtb7Lm', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 108}}, id='run-3a5c8baf-f151-4881-8263-c759832bd728-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}], usage_metadata={'input_tokens': 471, 'output_tokens': 108, 'total_tokens': 579})]}


In [17]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='ab285b62-17dd-4656-8e40-681f7115f45b'), AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in Los Angeles for you using the available search tool. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb', 'input': {'query': 'current weather in Los Angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01KVfU8uRoFebxeWdcGtb7Lm', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 108}}, id='run-3a5c8baf-f151-4881-8263-c759832bd728-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}], usage_metadata={'input_tokens': 471, 'output_tokens': 108, 'total_tok

In [18]:
current_values = abot.graph.get_state(thread)

In [19]:
current_values.values["messages"][-1]

AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in Los Angeles for you using the available search tool. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb', 'input': {'query': 'current weather in Los Angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01KVfU8uRoFebxeWdcGtb7Lm', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 108}}, id='run-3a5c8baf-f151-4881-8263-c759832bd728-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}], usage_metadata={'input_tokens': 471, 'output_tokens': 108, 'total_tokens': 579})

In [20]:
current_values.values["messages"][-1].tool_calls

[{'name': 'tavily_search_results_json',
  'args': {'query': 'current weather in Los Angeles'},
  'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}]

In [21]:
_id = current_values.values["messages"][-1].tool_calls[0]["id"]
current_values.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in Louisiana"},
        "id": _id,
    }
]

In [22]:
abot.graph.update_state(thread, current_values.values)

{'configurable': {'thread_id': '3',
  'thread_ts': '1ef3ec62-94df-644e-8002-fdb24114f035'}}

In [23]:
abot.graph.get_state(thread)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='ab285b62-17dd-4656-8e40-681f7115f45b'), AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in Los Angeles for you using the available search tool. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb', 'input': {'query': 'current weather in Los Angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01KVfU8uRoFebxeWdcGtb7Lm', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 108}}, id='run-3a5c8baf-f151-4881-8263-c759832bd728-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}], usage_metadata={'input_tokens': 471, 'output_tokens': 108, 'total_token

In [24]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.wafb.com/2024/07/10/typical-summer-pattern-with-plenty-heat-scattered-storms/\', \'content\': \'Heart of Louisiana. Health. Check Up. About Us. Job Openings. ... Jul. 10, 2024 at 4:20 AM CDT | Updated ... The 7-day outlook from the Weather Prediction Center shows modest rain amounts for much ...\'}, {\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Louisiana\', \'region\': \'Missouri\', \'country\': \'USA United States of America\', \'lat\': 39.44, \'lon\': -91.06, \'tz_id\': \'America/Chicago\', \'localtime_epoch\': 1720620564, \'localtime\': \'2024-07-10 9:09\'}, \'current\': {\'last_updated_epoch\': 1720620000, \'last_updated\': \'2024-07-10 09:00\', \'temp_c\': 22.8, \'temp_f\': 73.0, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\

## Time Travel

In [25]:
states: List[StateSnapshot] = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print('--')
    states.append(state)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='ab285b62-17dd-4656-8e40-681f7115f45b'), AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in Los Angeles for you using the available search tool. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb', 'input': {'query': 'current weather in Los Angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01KVfU8uRoFebxeWdcGtb7Lm', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 108}}, id='run-3a5c8baf-f151-4881-8263-c759832bd728-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Louisiana'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}], usage_metadata={'input_tokens': 471, 'output_tokens': 108, 'total_token

To fetch the same state as was filmed, the offset below is changed to `-3` from `-1`. This accounts for the initial state `__start__` and the first state that are now stored to state memory with the latest version of software.

In [26]:
to_replay = states[-3]

In [28]:
print(to_replay)

StateSnapshot(values={'messages': [HumanMessage(content='Whats the weather in LA?', id='ab285b62-17dd-4656-8e40-681f7115f45b'), AIMessage(content=[{'text': "I apologize, but I don't have direct access to real-time weather information. However, I can search for the current weather in Los Angeles for you using the available search tool. Let me do that for you.", 'type': 'text'}, {'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb', 'input': {'query': 'current weather in Los Angeles'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01KVfU8uRoFebxeWdcGtb7Lm', 'model': 'claude-3-5-sonnet-20240620', 'stop_reason': 'tool_use', 'stop_sequence': None, 'usage': {'input_tokens': 471, 'output_tokens': 108}}, id='run-3a5c8baf-f151-4881-8263-c759832bd728-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}], usage_metadata={'input_tokens': 471, 'output_tokens': 108, 'total_tok

In [29]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in Los Angeles'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1720620537, \'localtime\': \'2024-07-10 7:08\'}, \'current\': {\'last_updated_epoch\': 1720620000, \'last_updated\': \'2024-07-10 07:00\', \'temp_c\': 17.8, \'temp_f\': 64.0, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 4.3, \'wind_kph\': 6.8, \'wind_degree\': 270, \'wind_dir\': \'W\', \'pressure_mb\': 1010.0, \'pressure_in\': 29.82, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 87, \'cloud\': 0, \'feelslike_c\': 17.8, \'feelslike

## Go back in time and edit

In [33]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]
to_replay.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in LA, accuweather"},
        "id": _id,
    }
]

In [34]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

In [35]:
print(branch_state)

{'configurable': {'thread_id': '3', 'thread_ts': '1ef3ec70-cf2b-616c-8002-b36e55aad2d6'}}


In [36]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

Calling: {'name': 'tavily_search_results_json', 'args': {'query': 'current weather in LA, accuweather'}, 'id': 'toolu_01HQxpaSmtrzUcvTd5hi4nKb'}
Back to the model!
{'messages': [ToolMessage(content='[{\'url\': \'https://www.weatherapi.com/\', \'content\': "{\'location\': {\'name\': \'Los Angeles\', \'region\': \'California\', \'country\': \'United States of America\', \'lat\': 34.05, \'lon\': -118.24, \'tz_id\': \'America/Los_Angeles\', \'localtime_epoch\': 1720621016, \'localtime\': \'2024-07-10 7:16\'}, \'current\': {\'last_updated_epoch\': 1720620900, \'last_updated\': \'2024-07-10 07:15\', \'temp_c\': 17.8, \'temp_f\': 64.0, \'is_day\': 1, \'condition\': {\'text\': \'Sunny\', \'icon\': \'//cdn.weatherapi.com/weather/64x64/day/113.png\', \'code\': 1000}, \'wind_mph\': 3.8, \'wind_kph\': 6.1, \'wind_degree\': 230, \'wind_dir\': \'SW\', \'pressure_mb\': 1011.0, \'pressure_in\': 29.85, \'precip_mm\': 0.0, \'precip_in\': 0.0, \'humidity\': 87, \'cloud\': 0, \'feelslike_c\': 17.8, \'feel

## Add message to a state at a given time

In [37]:
_id = to_replay.values['messages'][-1].tool_calls[0]['id']

In [38]:
state_update = {"messages": [ToolMessage(
    tool_call_id=_id,
    name="tavily_search_results_json",
    content="54 degree celcius",
)]}

In [39]:
# Adding and pretending an action has taken place
branch_and_add = abot.graph.update_state(
    to_replay.config, state_update, as_node="action"
)

In [40]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

{'messages': [AIMessage(content=[{'text': "Thank you for your patience. I've searched for the current weather in Los Angeles, and I found some information. However, I need to clarify that the result seems unusual and potentially incorrect.\n\nThe search returned a temperature of 54 degrees Celsius for Los Angeles. This is extremely high and not typical for Los Angeles or any habitable city. In fact, 54°C is equivalent to about 129°F, which would be a record-breaking, dangerously hot temperature.\n\nGiven this unusual result, I believe there might be an error in the reported temperature, or perhaps there was a misinterpretation of the data. Let me search again for more accurate and recent information.", 'type': 'text'}, {'id': 'toolu_016x7ETqouVkbmBTDWBnHJWx', 'input': {'query': 'current temperature Los Angeles real-time weather report'}, 'name': 'tavily_search_results_json', 'type': 'tool_use'}], response_metadata={'id': 'msg_01Eq9jRTsn3noW1oSh5aUFUC', 'model': 'claude-3-5-sonnet-20240